# Ranking do PISA em Matemática, Leitura e Ciências (2003 à 2018)

## 1 - Importando pacotes

In [1]:
# Imports
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import basedosdados as bd
from plotly.subplots import make_subplots
import emoji
import requests
import datetime


import warnings
warnings.filterwarnings('ignore')

## 2 - Extração e pré-processamento

In [2]:
query = '''SELECT year, country_id_iso_3, AVG (score_mathematics) AS media_matematica, AVG (score_reading) AS media_leitura, AVG (score_science) AS media_ciencia
FROM `basedosdados.world_oecd_pisa.student_summary`
GROUP BY year, country_id_iso_3'''
df = bd.read_sql(query, billing_project_id='analise2-rgs')

Downloading: 100%|███████████████████████████████████████████████████████████████| 433/433 [00:00<00:00, 1155.11rows/s]


## 3 - Análise Exploratória

In [3]:
df.head()

,year,country_id_iso_3,media_matematica,media_leitura,media_ciencia
0,2000,ALB,398.372743,356.287861,379.011840
1,2000,ARG,396.997101,425.588670,405.300249
2,2000,AUS,527.591578,527.099479,524.970832
3,2000,AUT,503.207846,498.820858,509.310150
4,2000,BEL,517.627876,517.334637,500.653342


In [4]:
df.shape

(433, 5)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 433 entries, 0 to 432
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   year              433 non-null    Int64  
 1   country_id_iso_3  433 non-null    object 
 2   media_matematica  432 non-null    float64
 3   media_leitura     430 non-null    float64
 4   media_ciencia     432 non-null    float64
dtypes: Int64(1), float64(3), object(1)
memory usage: 17.5+ KB


In [6]:
df.isnull().sum()

year                0
country_id_iso_3    0
media_matematica    1
media_leitura       3
media_ciencia       1
dtype: int64

In [7]:
df.nunique()

year                  7
country_id_iso_3    100
media_matematica    432
media_leitura       430
media_ciencia       432
dtype: int64

In [8]:
years = df['year'].unique()

## 4 - Transformação

In [9]:
df_filtered = pd.DataFrame()

for country in df['country_id_iso_3'].unique():
    country_data = df[df['country_id_iso_3'] == country]
    
    missing_years = set(years) - set(country_data['year'])
    
    if len(missing_years) == 0:
        df_filtered = df_filtered.append(country_data)

df = df_filtered

In [10]:
df_filtered.shape

(238, 5)

In [11]:
df_filtered['country_id_iso_3'].unique()

array(['AUS', 'AUT', 'BEL', 'BRA', 'CAN', 'CHE', 'CZE', 'DEU', 'DNK',
       'ESP', 'FIN', 'FRA', 'GBR', 'GRC', 'HKG', 'HUN', 'IDN', 'IRL',
       'ISL', 'ITA', 'JPN', 'KOR', 'LUX', 'LVA', 'MEX', 'NLD', 'NOR',
       'NZL', 'POL', 'PRT', 'RUS', 'SWE', 'THA', 'USA'], dtype=object)

In [17]:
df_filtered['year'].unique()

array([2000, 2003, 2006, 2009, 2012, 2015, 2018], dtype=int64)

In [18]:
df_filtered[['media_matematica','media_leitura','media_ciencia']] = df_filtered[['media_matematica','media_leitura','media_ciencia']].round(2)

In [19]:
df_filtered['year'] = pd.to_datetime(df_filtered['year'], format='%Y').dt.year

In [20]:
df_filtered = df_filtered.sort_values(by=['year','country_id_iso_3'])

## 5 - Criação de Gráficos e Visualização

In [29]:
# Dados
countries = ['AUS', 'BRA', 'USA', 'DEU', 'ITA', 'MEX', 'PRT', 'RUS', 'THA']
df_plot = df[df['country_id_iso_3'].isin(countries)]

# Mapeia país para emoji de bandeira
country_flags = {
    'AUS': emoji.emojize(':Australia:'),
    'BRA': emoji.emojize(':Brazil:'),
    'USA': emoji.emojize(':United_States:'),
    'DEU': emoji.emojize(':Germany:'),  
    'ITA': emoji.emojize(':Italy:'),
    'MEX': emoji.emojize(':Mexico:'),
    'PRT': emoji.emojize(':Portugal:'),
    'RUS': emoji.emojize(':Russia:'),       
    'THA': emoji.emojize(':Thailand:')
}

# Anos disponíveis
years = df_plot['year'].unique()

# Paleta de cores personalizada
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#17becf', '#7f7f7f']

# Cria o gráfico de linhas
figm = go.Figure()

for i, country in enumerate(countries):
    df_country = df_plot[df_plot['country_id_iso_3'] == country]  
    name = country_flags[country]
        
    figm.add_trace(
        go.Scatter(
            x=df_country['year'].sort_values(),
            y=df_country['media_matematica'],
            name=name,
            legendgroup=country,
            line=dict(width=2, color=colors[i % len(colors)]),  # Aplica a cor da paleta
            mode='lines+markers',
            marker=dict(size=8, color=colors[i % len(colors)]),  # Aplica a cor da paleta aos marcadores
            showlegend=True
        )
    )

figm.update_layout(
    title='<b>Série Histórica do Ranking em Matemática (PISA)',
    title_x=0.5,
    yaxis_title='Pontuação',
    xaxis=dict(tickvals=years, tickmode='array'),
    plot_bgcolor='rgb(240, 240, 240)',
    paper_bgcolor='white',
    xaxis_gridcolor='lightgray',
    yaxis_gridcolor='lightgray',
    legend=dict(
        x=1.01,
        y=0.99,
        bgcolor='rgba(255, 255, 255, 0.8)',
        bordercolor='rgba(0, 0, 0, 0.2)',
        borderwidth=1
    )
)

figm.show()

In [30]:
# Cria o gráfico de linhas
figl = go.Figure()

for i, country in enumerate(countries):
    df_country = df_plot[df_plot['country_id_iso_3'] == country]  
    name = country_flags[country]
        
    figl.add_trace(
        go.Scatter(
            x=df_country['year'].sort_values(),
            y=df_country['media_leitura'],
            name=name,
            legendgroup=country,
            line=dict(width=2, color=colors[i % len(colors)]),  # Aplica a cor da paleta
            mode='lines+markers',
            marker=dict(size=8, color=colors[i % len(colors)]),  # Aplica a cor da paleta aos marcadores
            showlegend=True
        )
    )

figl.update_layout(
    title='<b>Série Histórica do Ranking em Leitura (PISA)',
    title_x=0.5,
    yaxis_title='Pontuação',
    xaxis=dict(tickvals=years, tickmode='array'),
    plot_bgcolor='rgb(240, 240, 240)',
    paper_bgcolor='white',
    xaxis_gridcolor='lightgray',
    yaxis_gridcolor='lightgray',
    legend=dict(
        x=1.01,
        y=0.99,
        bgcolor='rgba(255, 255, 255, 0.8)',
        bordercolor='rgba(0, 0, 0, 0.2)',
        borderwidth=1
    )
)

figl.show()

In [34]:
# Cria o gráfico de linhas
figc = go.Figure()

for i, country in enumerate(countries):
    df_country = df_plot[df_plot['country_id_iso_3'] == country]  
    name = country_flags[country]
        
    figc.add_trace(
        go.Scatter(
            x=df_country['year'].sort_values(),
            y=df_country['media_ciencia'],
            name=name,
            legendgroup=country,
            line=dict(width=2, color=colors[i % len(colors)]),  # Aplica a cor da paleta
            mode='lines+markers',
            marker=dict(size=8, color=colors[i % len(colors)]),  # Aplica a cor da paleta aos marcadores
            showlegend=True
        )
    )

figc.update_layout(
    title='<b>Série Histórica do Ranking em Ciência (PISA)',
    title_x=0.5,
    yaxis_title='Pontuação',
    xaxis=dict(tickvals=years, tickmode='array'),
    plot_bgcolor='rgb(240, 240, 240)',
    paper_bgcolor='white',
    xaxis_gridcolor='lightgray',
    yaxis_gridcolor='lightgray',
    legend=dict(
        x=1.01,
        y=0.99,
        bgcolor='rgba(255, 255, 255, 0.8)',
        bordercolor='rgba(0, 0, 0, 0.2)',
        borderwidth=1
    )
)

figc.show()

In [35]:
figm.write_image("mpisa.png", width=1200, height=800)
figl.write_image("lpisa.png", width=1200, height=800)
figc.write_image("cpisa.png", width=1200, height=800)